In [25]:
import pandas as pd
import numpy as np  
from sklearn.impute import SimpleImputer  # Gestion des valeurs manquantes
from sklearn.model_selection import train_test_split  # Division train/test
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Normalisation et encodage
# Gestion du déséquilibre de classes (alternative manuelle)
from sklearn.utils import resample  # Pour le resampling manuel


import warnings
warnings.filterwarnings('ignore')

In [2]:
#Chargement
df = pd.read_csv("Customer_Churn.csv")

# PRÉTRAITEMENT ET ENGINEERING DES FEATURES

In [7]:
# Création d'une copie pour le prétraitement
df_processed = df.copy()

#  Suppression de la colonne customerID (non pertinente pour la prédiction)
print("\n Suppression de la colonne 'customerID'")
df_processed = df_processed.drop('customerID', axis=1)

# Conversion de TotalCharges en numérique (peut contenir des espaces)
df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')

# Encodage temporaire de la variable cible
df_processed.Churn = df_processed.Churn.replace({'Yes': 1, 'No': 0})



 Suppression de la colonne 'customerID'


In [11]:
#  Gestion des valeurs manquantes dans TotalCharges
print(f"\n Gestion des valeurs manquantes dans TotalCharges")
print(f"  Nombre de valeurs manquantes : {df_processed['TotalCharges'].isna().sum()}")

# Imputation par la médiane (plus robuste aux outliers)
imputer = SimpleImputer(strategy='median')
df_processed['TotalCharges'] = imputer.fit_transform(df_processed[['TotalCharges']])

print(f"  Valeurs manquantes imputées par la médiane")


 Gestion des valeurs manquantes dans TotalCharges
  Nombre de valeurs manquantes : 11
  Valeurs manquantes imputées par la médiane


In [12]:
df_processed.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [13]:
#  Feature Engineering : Création de nouvelles variables
print("\n Création de nouvelles features (Feature Engineering) :")

# 1. Charges moyennes par mois (indicateur de valeur client)
df_processed['AvgChargesPerMonth'] = df_processed['TotalCharges'] / (df_processed['tenure'] + 1)
print("  - AvgChargesPerMonth : Charges moyennes mensuelles")

# 2. Indicateur de client senior
df_processed['IsSenior'] = df_processed['SeniorCitizen']
print("  - IsSenior : Indicateur senior")

# 3. Indicateur de services multiples
df_processed['HasMultipleServices'] = (
    (df_processed['PhoneService'] == 'Yes') & 
    (df_processed['InternetService'] != 'No')
).astype(int)
print("  - HasMultipleServices : Client avec téléphone ET internet")

# 4. Score de services premium (streaming, sécurité, etc.)
premium_services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                    'TechSupport', 'StreamingTV', 'StreamingMovies']
df_processed['PremiumServiceCount'] = df_processed[premium_services].apply(
    lambda row: sum([1 for val in row if val == 'Yes']), axis=1
)
print("  - PremiumServiceCount : Nombre de services premium souscrits")

# 5. Indicateur de contrat à risque (month-to-month)
df_processed['IsMonthToMonth'] = (df_processed['Contract'] == 'Month-to-month').astype(int)
print("  - IsMonthToMonth : Contrat mensuel (plus risqué)")



 Création de nouvelles features (Feature Engineering) :
  - AvgChargesPerMonth : Charges moyennes mensuelles
  - IsSenior : Indicateur senior
  - HasMultipleServices : Client avec téléphone ET internet
  - PremiumServiceCount : Nombre de services premium souscrits
  - IsMonthToMonth : Contrat mensuel (plus risqué)


In [14]:
#  Encodage des variables catégorielles
print("\n Encodage des variables catégorielles :")

# Mise à jour de la liste des variables catégorielles après feature engineering
categorical_features_updated = df_processed.select_dtypes(include=['object']).columns.tolist()
print(f"  Variables à encoder : {categorical_features_updated}")



 Encodage des variables catégorielles :
  Variables à encoder : ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [15]:
# Encodage One-Hot pour les variables nominales
# One-Hot crée une colonne binaire pour chaque catégorie
df_processed = pd.get_dummies(df_processed, columns=categorical_features_updated, drop_first=True)
print(f"   Encodage One-Hot effectué. Nouvelles dimensions : {df_processed.shape}")


   Encodage One-Hot effectué. Nouvelles dimensions : (7043, 36)


In [16]:
df_processed.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,AvgChargesPerMonth,IsSenior,HasMultipleServices,PremiumServiceCount,IsMonthToMonth,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,14.925000,0,0,1,1,...,False,False,False,False,False,False,True,False,True,False
1,0,34,56.95,1889.50,0,53.985714,0,1,2,0,...,False,False,False,False,True,False,False,False,False,True
2,0,2,53.85,108.15,1,36.050000,0,1,2,1,...,False,False,False,False,False,False,True,False,False,True
3,0,45,42.30,1840.75,0,40.016304,0,0,3,0,...,False,False,False,False,True,False,False,False,False,False
4,0,2,70.70,151.65,1,50.550000,0,1,0,1,...,False,False,False,False,False,False,True,False,True,False


In [17]:
# 4.6 Normalisation des variables numériques
print("\n Normalisation des variables numériques :")

# Séparation des features (X) et de la cible (y)
X = df_processed.drop('Churn', axis=1)
y = df_processed['Churn']

print(f"  Shape de X (features) : {X.shape}")
print(f"  Shape de y (cible) : {y.shape}")



 Normalisation des variables numériques :
  Shape de X (features) : (7043, 35)
  Shape de y (cible) : (7043,)


In [18]:
# Division en ensembles d'entraînement (80%) et de test (20%)
# random_state=42 pour la reproductibilité
# stratify=y pour conserver les mêmes proportions de Churn dans train et test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n✓ Division train/test effectuée :")
print(f"  X_train : {X_train.shape}")
print(f"  X_test  : {X_test.shape}")
print(f"  y_train : {y_train.shape}")
print(f"  y_test  : {y_test.shape}")


✓ Division train/test effectuée :
  X_train : (5634, 35)
  X_test  : (1409, 35)
  y_train : (5634,)
  y_test  : (1409,)


In [21]:
# Normalisation des features avec StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)       

print(f"\n Normalisation effectuée avec StandardScaler")



 Normalisation effectuée avec StandardScaler


In [22]:
# c) Sauvegarder les datasets préparés
pd.DataFrame(X_train_scaled).to_csv('X_train_prepared.csv', index=False)
pd.DataFrame(X_test_scaled).to_csv('X_test_prepared.csv', index=False)
y_train.to_csv('y_train_prepared.csv', index=False)
y_test.to_csv('y_test_prepared.csv', index=False)


In [26]:
# GESTION DU DÉSÉQUILIBRE DES CLASSES AVEC SMOTE

print("\n\n GESTION DU DÉSÉQUILIBRE DES CLASSES")
print("=" * 80)

# Over-sampling manuel de la classe minoritaire (alternative à SMOTE)
# Identifie la classe minoritaire et la sur-échantillonne par duplication

print("\n Application d'over-sampling pour rééquilibrer les classes :")
print(f"  Avant over-sampling - Classe 0 : {sum(y_train == 0)}, Classe 1 : {sum(y_train == 1)}")

# Séparer les classes
X_train_class_0 = X_train_scaled[y_train == 0]
X_train_class_1 = X_train_scaled[y_train == 1]
y_train_class_0 = y_train[y_train == 0]
y_train_class_1 = y_train[y_train == 1]

# Sur-échantillonner la classe minoritaire
if len(y_train_class_1) < len(y_train_class_0):
    # Classe 1 est minoritaire - la dupliquer
    X_train_class_1_upsampled = resample(X_train_class_1,
                                         replace=True,  # Échantillonnage avec remplacement
                                         n_samples=len(X_train_class_0),
                                         random_state=42)
    y_train_class_1_upsampled = resample(y_train_class_1,
                                         replace=True,
                                         n_samples=len(y_train_class_0),
                                         random_state=42)
    
    # Combiner les classes
    X_train_resampled = np.vstack([X_train_class_0, X_train_class_1_upsampled])
    y_train_resampled = np.concatenate([y_train_class_0, y_train_class_1_upsampled])
else:
    # Classe 0 est minoritaire
    X_train_class_0_upsampled = resample(X_train_class_0,
                                         replace=True,
                                         n_samples=len(X_train_class_1),
                                         random_state=42)
    y_train_class_0_upsampled = resample(y_train_class_0,
                                         replace=True,
                                         n_samples=len(X_train_class_1),
                                         random_state=42)
    
    # Combiner les classes
    X_train_resampled = np.vstack([X_train_class_0_upsampled, X_train_class_1])
    y_train_resampled = np.concatenate([y_train_class_0_upsampled, y_train_class_1])

print(f"  Après over-sampling - Classe 0 : {sum(y_train_resampled == 0)}, Classe 1 : {sum(y_train_resampled == 1)}")
print(f" Classes parfaitement équilibrées!")




 GESTION DU DÉSÉQUILIBRE DES CLASSES

 Application d'over-sampling pour rééquilibrer les classes :
  Avant over-sampling - Classe 0 : 4139, Classe 1 : 1495
  Après over-sampling - Classe 0 : 4139, Classe 1 : 4139
 Classes parfaitement équilibrées!
